# 🎵 Apollo Audio Restoration — Fork 5 (Unified)

*Original work: [Apollo: Band-sequence Modeling for High-Quality Music Restoration in Compressed Audio](https://github.com/JusperLee/Apollo)*

The Apollo model was trained to restore and enhance lossy MP3 audio (≤ 128 kbps).

---

### What's in Fork 5?

This notebook merges the best features from four community forks into one unified implementation:

| Feature | Source |
|---|---|
| Core chunked overlap-windowed inference engine | Fork 1 · jarredou |
| Configurable sample rate, output bit-depth, gain staging, fade size, CUDA device | Fork 2 · ibratabian17 |
| Directory batch mode, multi-format input, shlex-safe path handling | Fork 3 · Losses |
| `weights_only=False` PyTorch ≥ 2.0 fix, Baicai1145 Vocal MSST model, on-demand downloads | Fork 4 · Qupci |

---

### Available Models

| Model | Best for | Recommended chunk_size |
|---|---|---|
| **MP3 Enhancer** | General lossy audio restoration | 25 |
| **Lew Vocal Enhancer v1** | Separated vocal stems | 25 |
| **Lew Vocal Enhancer v2 (beta)** | Separated vocal stems (improved) | 25 |
| **Lew Universal Lossy Enhancer** | Mixed music & vocals | 19 |
| **Baicai1145 Vocal MSST** | Separated vocal stems (community model) | 10–15 |

---

### Quick Start

1. Run **Cell 1** to install dependencies and patch the Apollo repo.
2. *(Optional)* Run **Cell 2** to mount Google Drive.
3. Run **Cell 3** to download only the model(s) you want.
4. Run **Cell 4** to process a single file **or** run **Cell 5** for batch directory processing.

In [ ]:
#@title ⚙️ Cell 1 — Install Dependencies & Patch Apollo Repo
#@markdown Run this cell first. It clones the Apollo repository, applies the
#@markdown `base_model.py` patch (PyTorch ≥ 2.0 compatibility + Baicai checkpoint
#@markdown support), replaces `inference.py` with the Fork 5 unified version,
#@markdown and pins PyTorch to a known-good version.
#@markdown
#@markdown **Estimated time: ~3–5 minutes** (PyTorch reinstall is the slow part).

import subprocess, sys, os

def run(cmd, **kwargs):
    """Run a shell command, stream output, and raise on failure."""
    result = subprocess.run(cmd, shell=True, **kwargs)
    if result.returncode != 0:
        raise RuntimeError(f"Command failed (exit {result.returncode}): {cmd}")

print("[1/6] Cloning JusperLee/Apollo...")
run("git clone --quiet https://github.com/JusperLee/Apollo.git /content/Apollo")

print("[2/6] Creating model and config directories...")
os.makedirs("/content/Apollo/model", exist_ok=True)
os.makedirs("/content/Apollo/configs", exist_ok=True)
os.makedirs("/content/input_audio", exist_ok=True)
os.makedirs("/content/output_audio", exist_ok=True)

print("[3/6] Applying base_model.py patch (PyTorch ≥ 2.0 fix + Baicai support)...")
run(
    "wget -q 'https://raw.githubusercontent.com/Qupci/Apollo-Colab-Inference/main/base_model.py' "
    "-O /content/Apollo/look2hear/models/base_model.py"
)

print("[4/6] Installing Fork 5 unified inference.py...")
run(
    "wget -q 'https://raw.githubusercontent.com/YOUR_USERNAME/Apollo-Colab-Fork5/main/inference.py' "
    "-O /content/Apollo/inference.py"
)
# ↑ Replace YOUR_USERNAME with your GitHub username after uploading inference.py

print("[5/6] Installing Python dependencies...")
run(f"{sys.executable} -m pip install -q omegaconf ml_collections")

print("[6/6] Pinning PyTorch to 2.5.1 + CUDA 12.4 for compatibility...")
run(f"{sys.executable} -m pip uninstall -q -y torch torchvision torchaudio")
run(
    f"{sys.executable} -m pip install -q "
    "torch==2.5.1 torchvision torchaudio "
    "--index-url https://download.pytorch.org/whl/cu124"
)

print("\n✅ Setup complete! Default audio folders:")
print("   Input  → /content/input_audio")
print("   Output → /content/output_audio")

In [ ]:
#@title 📂 Cell 2 — Mount Google Drive  (optional)
#@markdown Mount your Google Drive if your audio files are stored there,
#@markdown or if you want to save outputs directly to Drive.
#@markdown After mounting, your Drive is available at `/content/drive/MyDrive/`.
#@markdown
#@markdown **Skip this cell** if you are uploading files directly to Colab.

from google.colab import drive
drive.mount("/content/drive")
print("✅ Google Drive mounted at /content/drive/MyDrive/")

In [ ]:
#@title 📥 Cell 3 — Download Models
#@markdown Select which models to download. Only the checked models will be
#@markdown fetched. You can re-run this cell at any time to add more models.
#@markdown Downloads are skipped if the file already exists.

import os, subprocess

# ── Model selection ────────────────────────────────────────────────────────
download_mp3_enhancer          = True  #@param {type:"boolean"}
download_lew_vocal_v1          = True  #@param {type:"boolean"}
download_lew_vocal_v2          = True  #@param {type:"boolean"}
download_lew_universal         = True  #@param {type:"boolean"}
download_baicai_vocal_msst     = False #@param {type:"boolean"}

# ── Download helper ────────────────────────────────────────────────────────
def download(url, dest_path, label):
    """Download `url` to `dest_path` using wget with resume support.
    Skips the download if the file already exists and is non-empty."""
    if os.path.exists(dest_path) and os.path.getsize(dest_path) > 0:
        print(f"  ⏭  {label} — already downloaded, skipping.")
        return
    os.makedirs(os.path.dirname(dest_path), exist_ok=True)
    print(f"  ⬇  {label}...")
    result = subprocess.run(
        ["wget", "-q", "-c", "--show-progress", "-O", dest_path, url]
    )
    if result.returncode != 0:
        print(f"  ❌  Failed to download {label} (exit {result.returncode})")
    else:
        print(f"  ✅  {label} saved to {dest_path}")

MODEL_DIR  = "/content/Apollo/model"
CONFIG_DIR = "/content/Apollo/configs"

print("Downloading selected models...\n")

if download_mp3_enhancer:
    download(
        "https://huggingface.co/JusperLee/Apollo/resolve/main/pytorch_model.bin",
        f"{MODEL_DIR}/pytorch_model.bin",
        "MP3 Enhancer (pytorch_model.bin)"
    )

if download_lew_vocal_v1:
    download(
        "https://huggingface.co/jarredou/lew_apollo_vocal_enhancer/resolve/main/apollo_model.ckpt",
        f"{MODEL_DIR}/apollo_model.ckpt",
        "Lew Vocal Enhancer v1"
    )

if download_lew_vocal_v2:
    download(
        "https://huggingface.co/jarredou/lew_apollo_vocal_enhancer/resolve/main/apollo_model_v2.ckpt",
        f"{MODEL_DIR}/apollo_model_v2.ckpt",
        "Lew Vocal Enhancer v2 (beta)"
    )
    download(
        "https://huggingface.co/jarredou/lew_apollo_vocal_enhancer/resolve/main/config_apollo_vocal.yaml",
        f"{CONFIG_DIR}/config_apollo_vocal.yaml",
        "Config: config_apollo_vocal.yaml"
    )

if download_lew_universal:
    download(
        "https://github.com/deton24/Lew-s-vocal-enhancer-for-Apollo-by-JusperLee/releases/download/uni/apollo_model_uni.ckpt",
        f"{MODEL_DIR}/apollo_model_uni.ckpt",
        "Lew Universal Lossy Enhancer"
    )
    download(
        "https://github.com/deton24/Lew-s-vocal-enhancer-for-Apollo-by-JusperLee/releases/download/uni/config_apollo_uni.yaml",
        f"{CONFIG_DIR}/config_apollo_uni.yaml",
        "Config: config_apollo_uni.yaml"
    )

if download_baicai_vocal_msst:
    download(
        "https://huggingface.co/baicai1145/Apollo-vocal-msst/resolve/main/model_apollo_0129_-21.98.ckpt",
        f"{MODEL_DIR}/baicai_apollo_vocal_msst.ckpt",
        "Baicai1145 Vocal MSST"
    )
    download(
        "https://huggingface.co/baicai1145/Apollo-vocal-msst/resolve/main/config_apollo_0129_-21.98.yaml",
        f"{CONFIG_DIR}/config_baicai_vocal_msst.yaml",
        "Config: config_baicai_vocal_msst.yaml"
    )

print("\n✅ Model download step complete.")

In [ ]:
#@title 🎧 Cell 4 — Single File Inference
#@markdown Process one audio file. Supports WAV, MP3, FLAC, M4A, OPUS, OGG, AIFF.
#@markdown
#@markdown **Tip:** Upload your file to `/content/input_audio/` using the
#@markdown file browser on the left, then set `input_file` below.

import os, shlex, subprocess
%cd /content/Apollo

# ── I/O ────────────────────────────────────────────────────────────────────
input_file  = "/content/input_audio/input.wav"  #@param {type:"string"}
output_file = "/content/output_audio/output.wav" #@param {type:"string"}

# ── Model ──────────────────────────────────────────────────────────────────
model = "Lew Universal Lossy Enhancer"  #@param ["MP3 Enhancer", "Lew Vocal Enhancer v1", "Lew Vocal Enhancer v2 (beta)", "Lew Universal Lossy Enhancer", "Baicai1145 Vocal MSST"]

# ── Processing ─────────────────────────────────────────────────────────────
#@markdown ---
#@markdown #### Processing Settings
#@markdown *Use `chunk_size=19` for Lew Universal, `chunk_size=25` for other models.*
#@markdown *Use `chunk_size=10` for Baicai Vocal MSST.*
chunk_size = 19  #@param {type:"slider", min:3, max:25, step:1}
overlap    = 2   #@param {type:"slider", min:2, max:10, step:1}
fade_sec   = 3   #@param {type:"slider", min:1, max:10, step:1}
sample_rate = 44100  #@param {type:"number"}

# ── Audio Quality ──────────────────────────────────────────────────────────
#@markdown ---
#@markdown #### Audio Quality
out_subtype = "FLOAT"  #@param ["FLOAT", "PCM_24", "PCM_16"]
#@markdown `FLOAT` (32-bit) is recommended — it avoids clipping and preserves full dynamic range.
gain_in  = 0  #@param {type:"slider", min:-24, max:24, step:0.5}
gain_out = 0  #@param {type:"slider", min:-24, max:24, step:0.5}
#@markdown `gain_in` / `gain_out` adjust level in dB before / after inference.

# ── Hardware ───────────────────────────────────────────────────────────────
cuda_device = "0"  #@param {type:"string"}

# ── Resolve checkpoint & config ────────────────────────────────────────────
MODEL_DIR  = "/content/Apollo/model"
CONFIG_DIR = "/content/Apollo/configs"

model_map = {
    "MP3 Enhancer": {
        "ckpt":   f"{MODEL_DIR}/pytorch_model.bin",
        "config": f"{CONFIG_DIR}/apollo.yaml",
    },
    "Lew Vocal Enhancer v1": {
        "ckpt":   f"{MODEL_DIR}/apollo_model.ckpt",
        "config": f"{CONFIG_DIR}/apollo.yaml",
    },
    "Lew Vocal Enhancer v2 (beta)": {
        "ckpt":   f"{MODEL_DIR}/apollo_model_v2.ckpt",
        "config": f"{CONFIG_DIR}/config_apollo_vocal.yaml",
    },
    "Lew Universal Lossy Enhancer": {
        "ckpt":   f"{MODEL_DIR}/apollo_model_uni.ckpt",
        "config": f"{CONFIG_DIR}/config_apollo_uni.yaml",
    },
    "Baicai1145 Vocal MSST": {
        "ckpt":   f"{MODEL_DIR}/baicai_apollo_vocal_msst.ckpt",
        "config": f"{CONFIG_DIR}/config_baicai_vocal_msst.yaml",
    },
}

selected = model_map[model]
ckpt_path   = selected["ckpt"]
config_path = selected["config"]

# Validate that the required files exist
missing = [p for p in (ckpt_path, config_path) if not os.path.exists(p)]
if missing:
    raise FileNotFoundError(
        f"The following required files are missing:\n" +
        "\n".join(f"  {p}" for p in missing) +
        "\nPlease run Cell 3 to download the selected model first."
    )

if not os.path.exists(input_file):
    raise FileNotFoundError(f"Input file not found: {input_file}")

# Ensure output directory exists
os.makedirs(os.path.dirname(os.path.abspath(output_file)), exist_ok=True)

# ── Build & run command ────────────────────────────────────────────────────
cmd = (
    f"python inference.py "
    f"--in_wav  {shlex.quote(input_file)} "
    f"--out_wav {shlex.quote(output_file)} "
    f"--ckpt    {shlex.quote(ckpt_path)} "
    f"--config  {shlex.quote(config_path)} "
    f"--chunk_size {chunk_size} "
    f"--overlap    {overlap} "
    f"--fade_sec   {fade_sec} "
    f"--sr         {sample_rate} "
    f"--out_subtype {shlex.quote(out_subtype)} "
    f"--gain_in  {gain_in} "
    f"--gain_out {gain_out} "
    f"--cuda     {shlex.quote(cuda_device)}"
)

print(f"Model    : {model}")
print(f"Input    : {input_file}")
print(f"Output   : {output_file}")
print()
get_ipython().system(cmd)

In [ ]:
#@title 📁 Cell 5 — Batch Directory Inference
#@markdown Process **all** audio files in a directory.
#@markdown Supports: WAV, MP3, FLAC, M4A, OPUS, OGG, AIFF.
#@markdown All outputs are saved as WAV to the output directory.
#@markdown
#@markdown The model is loaded **once** and reused for every file,
#@markdown making this much more efficient than running Cell 4 repeatedly.
#@markdown Failed files are skipped with a warning rather than halting the batch.

import os, shlex
%cd /content/Apollo

# ── I/O ────────────────────────────────────────────────────────────────────
input_directory  = "/content/input_audio"   #@param {type:"string"}
output_directory = "/content/output_audio"  #@param {type:"string"}

# ── Model ──────────────────────────────────────────────────────────────────
model = "Lew Universal Lossy Enhancer"  #@param ["MP3 Enhancer", "Lew Vocal Enhancer v1", "Lew Vocal Enhancer v2 (beta)", "Lew Universal Lossy Enhancer", "Baicai1145 Vocal MSST"]

# ── Processing ─────────────────────────────────────────────────────────────
#@markdown ---
#@markdown #### Processing Settings
chunk_size  = 19   #@param {type:"slider", min:3, max:25, step:1}
overlap     = 2    #@param {type:"slider", min:2, max:10, step:1}
fade_sec    = 3    #@param {type:"slider", min:1, max:10, step:1}
sample_rate = 44100 #@param {type:"number"}

# ── Audio Quality ──────────────────────────────────────────────────────────
#@markdown ---
#@markdown #### Audio Quality
out_subtype = "FLOAT"  #@param ["FLOAT", "PCM_24", "PCM_16"]
gain_in  = 0  #@param {type:"slider", min:-24, max:24, step:0.5}
gain_out = 0  #@param {type:"slider", min:-24, max:24, step:0.5}

# ── Hardware ───────────────────────────────────────────────────────────────
cuda_device = "0"  #@param {type:"string"}

# ── Resolve checkpoint & config ────────────────────────────────────────────
MODEL_DIR  = "/content/Apollo/model"
CONFIG_DIR = "/content/Apollo/configs"

model_map = {
    "MP3 Enhancer": {
        "ckpt":   f"{MODEL_DIR}/pytorch_model.bin",
        "config": f"{CONFIG_DIR}/apollo.yaml",
    },
    "Lew Vocal Enhancer v1": {
        "ckpt":   f"{MODEL_DIR}/apollo_model.ckpt",
        "config": f"{CONFIG_DIR}/apollo.yaml",
    },
    "Lew Vocal Enhancer v2 (beta)": {
        "ckpt":   f"{MODEL_DIR}/apollo_model_v2.ckpt",
        "config": f"{CONFIG_DIR}/config_apollo_vocal.yaml",
    },
    "Lew Universal Lossy Enhancer": {
        "ckpt":   f"{MODEL_DIR}/apollo_model_uni.ckpt",
        "config": f"{CONFIG_DIR}/config_apollo_uni.yaml",
    },
    "Baicai1145 Vocal MSST": {
        "ckpt":   f"{MODEL_DIR}/baicai_apollo_vocal_msst.ckpt",
        "config": f"{CONFIG_DIR}/config_baicai_vocal_msst.yaml",
    },
}

selected    = model_map[model]
ckpt_path   = selected["ckpt"]
config_path = selected["config"]

# Validate model files
missing = [p for p in (ckpt_path, config_path) if not os.path.exists(p)]
if missing:
    raise FileNotFoundError(
        f"Missing required files:\n" +
        "\n".join(f"  {p}" for p in missing) +
        "\nPlease run Cell 3 to download the selected model first."
    )

if not os.path.isdir(input_directory):
    raise NotADirectoryError(f"Input directory not found: {input_directory}")

os.makedirs(output_directory, exist_ok=True)

# ── Build & run command ────────────────────────────────────────────────────
cmd = (
    f"python inference.py "
    f"--in_dir  {shlex.quote(input_directory)} "
    f"--out_dir {shlex.quote(output_directory)} "
    f"--ckpt    {shlex.quote(ckpt_path)} "
    f"--config  {shlex.quote(config_path)} "
    f"--chunk_size {chunk_size} "
    f"--overlap    {overlap} "
    f"--fade_sec   {fade_sec} "
    f"--sr         {sample_rate} "
    f"--out_subtype {shlex.quote(out_subtype)} "
    f"--gain_in  {gain_in} "
    f"--gain_out {gain_out} "
    f"--cuda     {shlex.quote(cuda_device)}"
)

print(f"Model  : {model}")
print(f"Input  : {input_directory}")
print(f"Output : {output_directory}")
print()
get_ipython().system(cmd)